# Linear models, loss functions, gradients, SGD

* Objectives: Define, train and visualize a simple model; understand gradients and SGD; learn to use the GPU.
* Prerequisites: [Callable objects](https://docs.julialang.org/en/v1/manual/methods/#Function-like-objects-1), MNIST data (02.mnist.ipynb)
* AutoGrad: Param, @diff, gradient, value (used and explained)
* Knet: accuracy, zeroone, nll, train! (defined and explained)
* Knet: gpu, KnetArray (used and explained)
* Knet: dir, minibatch (used by mnist.jl)
* Knet: load, save (used by the experiment)

In [ ]:
# Load data (see 02.mnist.ipynb)
using Knet: Knet, minibatch
include(Knet.dir("data","mnist.jl"))
dtrn,dtst = mnistdata(xsize=(784,:),xtype=Array{Float32});

## Define linear model

In [ ]:
# We will use a callable object to define our linear model 
struct Linear; w; b; end
(model::Linear)(x) = model.w * x .+ model.b

## Prediction and accuracy

In [ ]:
# Let's take the first minibatch from the test set
x,y = first(dtst)
summary.((x,y))

In [ ]:
# Initialize a random Linear model
model = Linear(randn(10,784)*0.01, zeros(10))

In [ ]:
# Display its prediction on the first minibatch: a 10xN score matrix
ENV["COLUMNS"]=92
ypred = model(x)

In [ ]:
# correct answers are given as an array of integers
y'

In [ ]:
# We can calculate the accuracy of our model for the first minibatch
using Statistics
accuracy(model,x,y) = mean(y' .== map(i->i[1], findmax(Array(model(x)),dims=1)[2]))
accuracy(model,x,y)

In [ ]:
# We can calculate the accuracy of our model for the whole test set
using Knet: Data  # type of dtrn and dtst
accuracy(model,data::Data) = mean(accuracy(model,x,y) for (x,y) in data)
accuracy(model,dtst)

In [ ]:
# ZeroOne loss (or error) is defined as 1 - accuracy
zeroone(x...) = 1 - accuracy(x...)
zeroone(model,dtst)

## Negative log likelihood

In [ ]:
# Negative log likelihood (aka cross entropy, softmax loss, NLL)
function nll(model, x, y)
    scores = model(x)
    expscores = exp.(scores)
    probabilities = expscores ./ sum(expscores, dims=1)
    answerprobs = (probabilities[y[i],i] for i in 1:length(y))
    mean(-log.(answerprobs))
end

In [ ]:
# Calculate NLL of our model for the first minibatch
nll(model,x,y)

In [ ]:
# per-instance average negative log likelihood for the whole test set
nll(model,data::Data) = mean(nll(model,x,y) for (x,y) in data)
nll(model,dtst)

## Calculating the gradient using AutoGrad

In [ ]:
using AutoGrad
@doc AutoGrad

In [ ]:
using Random
Random.seed!(9);

In [ ]:
# To compute gradients we need to mark fields of f as Params:
model = Linear(Param(randn(10,784)), Param(zeros(10)))

In [ ]:
# We can still do predictions with f and calculate loss:
nll(model,x,y)

In [ ]:
# And we can do the same loss calculation also computing gradients:
J = @diff nll(model,x,y)

In [ ]:
# To get the actual loss value from J:
value(J)

In [ ]:
# To get the gradient of a parameter from J:
∇w = gradient(J,model.w)

In [ ]:
# Note that each gradient has the same size and shape as the corresponding parameter:
∇b = gradient(J,model.b)

## Checking the gradient using numerical approximation

In [ ]:
# Meaning of gradient: If I move the last entry of f.b by epsilon, loss will go up by 0.792576 epsilon!
@show ∇b;

In [ ]:
@show model.b;

In [ ]:
nll(model,x,y)     # loss for the first minibatch with the original parameters

In [ ]:
model.b[10] = 0.1   # to numerically check the gradient let's move the last entry of f.b by +0.1.
@show model.b;

In [ ]:
nll(model,x,y)     # We see that the loss moves by ≈ +0.79*0.1 as expected.

In [ ]:
model.b[10] = 0

## Checking the gradient using manual implementation

In [ ]:
# Without AutoGrad we would have to define the gradients manually:
function nllgrad(model,x,y)
    scores = model(x)
    expscores = exp.(scores)
    probabilities = expscores ./ sum(expscores, dims=1)
    for i in 1:length(y); probabilities[y[i],i] -= 1; end
    dJds = probabilities / length(y)
    dJdw = dJds * x'
    dJdb = vec(sum(dJds,dims=2))
    dJdw,dJdb
end;

In [ ]:
∇w2,∇b2 = nllgrad(model,x,y)

In [ ]:
∇w2 ≈ ∇w

In [ ]:
∇b2 ≈ ∇b

## Training with Stochastic Gradient Descent (SGD)

In [ ]:
using LinearAlgebra: axpy!

In [ ]:
function train!(model, data; lr=0.1)
    for (x,y) in data
        loss = @diff Knet.nll(model,x,y)  # Knet.nll is bit more efficient
        for param in (model.w, model.b)
            ∇param = gradient(loss, param)
            axpy!(-lr, ∇param, value(param))
        end
    end
end

In [ ]:
# Let's try a randomly initialized model for 10 epochs
model = Linear(Param(randn(10,784)*0.01), Param(zeros(10)))
dtrn.xtype = dtst.xtype = Array{Float32}
@show nll(model,dtst)
@time for i=1:10; train!(model,dtrn); end
@show nll(model,dtst)

## Using the GPU

In [ ]:
# To work on the GPU, all we have to do is convert our Arrays to KnetArrays:
using Knet: KnetArray   # KnetArrays are allocated on and operated by the GPUs
if Knet.gpu() >= 0      # Knet.gpu() returns a device id >= 0 if there is a GPU, -1 otherwise
    ka = KnetArray{Float32}
    dtrn.xtype = dtst.xtype = ka
    model = Linear(Param(ka(randn(10,784)*0.01)), Param(ka(zeros(10))))
    @show nll(model,dtst)
    @time for i=1:10; train!(model,dtrn); end
    @show nll(model,dtst)
end

In [ ]:
# Let's collect some data to draw training curves and visualizing weights:
if !isfile("lin.jld2")
    models = []; trnloss = []; tstloss = []; trnerr = []; tsterr = []
    pa(x) = Knet.gpu() >= 0 ? Param(KnetArray{Float32}(x)) : Param(Array{Float32}(x))
    model = Linear(pa(randn(10,784)*0.01), pa(zeros(10)))
    @time while true
        push!(models, deepcopy(model))
        push!(trnloss, Knet.nll(model,dtrn))
        push!(tstloss, Knet.nll(model,dtst))
        push!(trnerr, zeroone(model,dtrn))
        push!(tsterr, zeroone(model,dtst))
        length(tsterr) == 100 && break
        train!(model,dtrn)
    end
    Knet.save("lin.jld2","models",models,"trnloss",trnloss,"tstloss",tstloss,"trnerr",trnerr,"tsterr",tsterr)
end

In [ ]:
# Use Knet.load and Knet.save to store models, results, etc.
lin = Knet.load("lin.jld2")
minimum(lin["tstloss"]), minimum(lin["tsterr"])

## Linear model shows underfitting

In [ ]:
using Plots
plot([lin["trnloss"], lin["tstloss"]],ylim=(.0,.4),labels=[:trnloss :tstloss],xlabel="Epochs",ylabel="Loss") 
# Demonstrates underfitting: training loss not close to 0
# Also slight overfitting: test loss higher than train

In [ ]:
plot([lin["trnerr"], lin["tsterr"]],ylim=(.0,.12),labels=[:trnerr :tsterr],xlabel="Epochs",ylabel="Error")  
# this is the error plot, we get to about 7.5% test error, i.e. 92.5% accuracy

## Visualizing the learned weights

In [ ]:
# Let us visualize the evolution of the weight matrix as images below
# Each row is turned into a 28x28 image with positive numbers light and negative numbers dark gray
using Images, ImageMagick
for t in 10 .^ range(0,stop=log10(length(lin["models"])),length=10) #logspace(0,2,20)
    i = floor(Int,t)
    f = lin["models"][i]
    w1 = reshape(Array(value(f.w))', (28,28,1,10))
    w2 = clamp.(w1.+0.5,0,1)
    IJulia.clear_output(true)
    display(hcat([mnistview(w2,i) for i=1:10]...))
    display("Epoch $i")
    sleep(1) # (0.96^i)
end